# Project 1
- **Team Members**: Chika Ozodiegwu, Kelsey Wyatt, Libardo Lambrano, Kurt Pessa

![](Images/florida_covid19_data.jpg)

### Data set used:: 
* https://open-fdoh.hub.arcgis.com/datasets/florida-covid19-case-line-data


##### Dependencies

In [1]:
import step1_raw_data_collection as step1
import step2_data_processing_and_cleaning as step2

import pandas as pd

## Step 1: Raw Data Collection
-----

In [2]:
df = step1.get_data()
help(step1.get_data)

Help on function get_data in module step1_raw_data_collection:

get_data(option='static')
    Returns a pandas dataframe with data about Florida cases
    
    Parameters:
        option (string): which data set to get
        
            options include: 
                1. "api and save"
                    -- requests new data and then saves/replaces as csv file in Resources
                2. "api"
                    -- just requests new data from api
                3. "static"
                    -- reads data from Resources folder
                Default: defaults to "static" dataset
    Returns:
        df (returns a dataframe of covid data)



## Step 2: Data Processing & Data Cleaning
-----

##### Single group

In [ ]:
#df = step2.get_hospitalized_data()
#df = step2.get_df_with_datetime_and_formatted_column()
#df = step2.get_hospitalizations_by_casedatetime()

#group_name = "Gender"
#group_name = "Age_group"
#group_name = "Travel_related"
#group_name = "Jurisdiction"
#group_name = "County"

df = step2.get_group(group_name)

df

##### Two groups: before and after opening

In [17]:
#df1, df2 = step2.get_groups_before_and_after_opening_date()
df1, df2 = step2.get_groups_by_casedatetime()

#group_name = "Gender"
#group_name = "Age_group"
#group_name = "Travel_related"
#group_name = "Jurisdiction"
#group_name = "County"

#df1,df2 = step2.get_groups(group_name)

pd.concat([df1,df2],axis=1)


,CaseDateTime,Hospitalized,CaseDateTime,Hospitalized
0,2020-04-04,234,2020-05-05,101
1,2020-04-05,162,2020-05-06,96
2,2020-04-06,249,2020-05-07,63
3,2020-04-07,221,2020-05-08,101
4,2020-04-08,244,2020-05-09,134
5,2020-04-09,256,2020-05-10,90
6,2020-04-10,255,2020-05-11,116
7,2020-04-11,186,2020-05-12,130
8,2020-04-12,186,2020-05-13,152
9,2020-04-13,214,2020-05-14,164


In [ ]:
#clean csv

new_csv_data_df = df[['ObjectId', "County",'Age',"Age_group", "Gender", "Jurisdiction", "Travel_related", "Hospitalized","Case1"]]
new_csv_data_df.head()

In [ ]:
#new_csv_data_df.to_csv(new_csv_data_df, 'new_cleaned_data.csv')

new_csv_data_df.to_csv ("new_covid_dataframe.csv") # index = False, header=True)

# There is no change in hospitalizations since reopening
### Research Question to Answer:
* “There is no change in hospitalizations since reopening?” 

### Part 1: Six (6) Steps for Hypothesis Testing

<details><summary> click to expand  </summary>

#### 1. Identify
- **Populations** (divide Hospitalization data in two groups of data):
    1. Prior to opening
    2. After opening  
* Decide on the **date**:
    * May 4th - restaurants opening to 25% capacity
    * June  (Miami opening beaches)
- Distribution:
    * Distribution

#### 2. State the hypotheses
- **H0**: There is no change in hospitalizations after Florida has reopened
- **H1**: There is a change in hospitalizations after Florida has reopened

#### 3. Characteristics of the comparison distribution
- Population means, standard deviations

#### 4. Critical values
- p = 0.05
- Our hypothesis is nondirectional so our hypothesis test is **two-tailed**

#### 5. Calculate

#### 6. Decide!
    
</details>

### Part 2: Visualization
* Trends

### Further Inquries
* Increases in groups?
* Age
* Gender
* Ethnicitiy

### Rough Breakdown of Tasks
* Data Massaging

In [ ]:
#Calculate total number of cases 
Total_covid_cases = new_csv_data_df["ObjectId"].nunique()
Total_covid_cases = pd.DataFrame({"Total Number of Cases": [Total_covid_cases]})
Total_covid_cases

In [ ]:
#Total number of cases by county (Kelsey) Include bar chart


In [ ]:
#Total number of cases by gender (Kelsey) Include pie chart 

In [ ]:
#Top 10 counties with the most cases (Libardo)


In [ ]:
#Divide hospitalization data in two groups of data prior to reopening and create new dataframe (Kurt) consider total (Chika)

In [ ]:
#Divide hospitalization data in two groups of data after reopening and create new dataframe (Kurt) condider total (Chika)

In [ ]:
#Total number of hospitalization for all counties (Libardo)

In [ ]:
#Total number of hospitalization for each county and put in DataFrame # Create a visualization (Kelsey)


In [ ]:
#Percentage of hospitalization by gender # Create Visualization (Libardo)


In [ ]:
#Percentage of hospitalization by age group (Chika) #Create visualization

In [ ]:
#Percentage of hospitalization before shut down (Not done yet) (Rephrase) (Chika)

In [ ]:
#Percentage of hospitalization during shut down (backburner)

In [ ]:
#Percentage of hospitalization after reopening(Not done yet) (Rephrase) (Chika)

In [ ]:
#Compare travel-related hospitalization to non-travelrelated cases (Not done yet) (Chika)

In [ ]:
#Average number of hospitalization by county (Not done yet) (Kelsey) 

In [ ]:
#Hospitalization by case date/month (needs more) (Libardo) 